In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import pandas as pd
from IPython.display import clear_output

In [11]:
df = pd.read_csv("/data/tim/heronWorkspace/ImageData/imagePropsSBU4.csv", on_bad_lines="warn")
df = df[df["grayscale"] == "False"].sort_values(by=['ImagePath'])
df.head(10)

,Unnamed: 0,cam,ImagePath,badImage,motion,grayscale
5223,5223,15.0,2017_SBU4_01270001,False,False,False
15224,15224,15.0,2017_SBU4_01270002,False,True,False
18724,18724,15.0,2017_SBU4_01270003,False,False,False
5150,5150,15.0,2017_SBU4_01270004,False,False,False
8334,8334,15.0,2017_SBU4_01270005,False,False,False
1043,1043,15.0,2017_SBU4_01270006,False,False,False
7698,7698,15.0,2017_SBU4_01270007,False,False,False
15114,15114,15.0,2017_SBU4_01280069,False,False,False
18917,18917,15.0,2017_SBU4_01280070,False,False,False
9496,9496,15.0,2017_SBU4_01280071,False,False,False


In [30]:
#load classified data from Tania
df = pd.read_csv("/data/tim/heronWorkspace/ImageData/imagePropsSBU4.csv", on_bad_lines="warn")
df = df[(df["grayscale"] == "False") & (df["badImage"] == "False")].sort_values(by=['ImagePath'])
dfSBU4 = pd.read_csv("/data/tim/heronWorkspace/datasetValidation.csv", on_bad_lines="warn")

try:
    currIndex = int(input("Enter index from last time: "))
except ValueError:
    print("not a number")
    
imgPropsList = []
for i, imgName in list(enumerate(df["ImagePath"]))[currIndex:]:
    img = Image.open(f'/data/shared/herons/TinaDubach_data/SBU4/{imgName}.JPG')
    fig = plt.imshow(img)
    plt.show()

    #enter number
    try:
        val = int(input("Enter a number: (0: normal image, 1: small anomaly heron: 2: big anomaly heron, 3: small anomaly other, 4: big anomaly other, -1: special image)"))
        imgPropsList.append([imgName, val])
    except ValueError:
        clear_output(wait=True)
        print("Quit program")
        print(f"Currently at image: {int(i/10) * 10}")
        
        # analyse current results
        camDataDf = pd.read_csv("/data/shared/herons/TinaDubach_data/CameraData_2017_july.csv", encoding='unicode_escape', on_bad_lines="warn", sep=";")
        df = pd.merge(camDataDf, dfSBU4, left_on="fotocode", how="right", right_on="ImagePath")
        nrFalsePos = len(df[(df["ValidationValue"] > 0) & (df["species"].isna())])
        nrFalseNeg = len(df[(df["ValidationValue"] == 0) & df["species"].notna()]) # always put conditions in brackets!
        nrTruePos = len(df[(df["ValidationValue"] > 0) & df["species"].notna()])
        nrTrueNeg = len(df[(df["ValidationValue"] == 0) & df["species"].isna()])
        nrNegNew = len( df[(df["ValidationValue"]==0)])
        # nrNegOld = len( df[(df["species"].isna()) | (df["fotocode"].isna())])
        nrNegOld = len( df[(df["species"].isna())])

        dfWithMoveFlag = pd.merge(df, dfSBU4, on="ImagePath", how="right")
        # nrFalseNegNoMove 

        print("Evaluation of tina dubachs classification with my classification as true value:")
        print(f"Nr of false positives (positive images now, that aren't in tina dubachs data, FP): {nrFalsePos}")
        print(f"Nr of false negatives (negative images now, that were in tina dubachs data, FN): {nrFalseNeg}")
        print(f"Nr of true positives, TP: {nrTruePos}")
        print(f"Nr of true negatives, TN: {nrTrueNeg}")
        
        print("Evalation of new data:")
        print(f"Nr of images classified as negative now: {nrNegNew}")
        print(f"Total Nr of images that aren't in tina dubachs positive data (were classified as negative before) : {nrNegOld}")

        # print(df[df["ValidationValue"] == 0 & df["species"].notna()][["species", "fotocode", "ImagePath", "ValidationValue"]])
        # print(df[["species", "fotocode", "ImagePath", "ValidationValue"]])
        break

    if i % 10 == 9:
        try:
            dfNew = pd.DataFrame(imgPropsList, columns=["ImagePath", "ValidationValue"])
            dfSBU4 = pd.concat([dfSBU4, dfNew])
            dfSBU4.to_csv("/data/tim/heronWorkspace/datasetValidation.csv")
            imgPropsList = []
            print(f"Saved file up to index:{i}")
        except Exception as e:
            print(e)
            print("Error saving file")
            print(f"Currently at image: {int(i/10) * 10}")
            break

    clear_output(wait=True)

Quit program
Currently at image: 240
Evaluation of tina dubachs classification with my classification as true value:
Nr of false positives (positive images now, that aren't in tina dubachs data, FP): 17
Nr of false negatives (negative images now, that were in tina dubachs data, FN): 0
Nr of true positives, TP: 19
Nr of true negatives, TN: 204
Evalation of new data:
Nr of images classified as negative now: 204
Total Nr of images that aren't in tina dubachs positive data (were classified as negative before) : 221


In [19]:
# df = df[df["species"].notna() & (df["ValidationValue"] == 0)]
# df = df[df["species"].notna()]
df[["species", "fotocode", "ImagePath", "ValidationValue"]]

,species,fotocode,ImagePath,ValidationValue
0,NaN,NaN,2017_SBU4_01270001,0
1,NaN,NaN,2017_SBU4_01270002,0
2,NaN,NaN,2017_SBU4_01270003,0
3,NaN,NaN,2017_SBU4_01270004,0
4,NaN,NaN,2017_SBU4_01270005,0
5,NaN,NaN,2017_SBU4_01270006,0
6,NaN,NaN,2017_SBU4_01270007,0
7,NaN,NaN,2017_SBU4_01280069,0
8,NaN,NaN,2017_SBU4_01280070,4
9,NaN,NaN,2017_SBU4_01280071,0
